# Density plots

In [1]:
library(raster)
library(RColorBrewer)
library(colorRamps)
library(viridis)

Warning message:
"Paket 'raster' wurde unter R Version 4.2.3 erstellt"
Lade nötiges Paket: sp

Warning message:
"Paket 'sp' wurde unter R Version 4.2.3 erstellt"
The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Warning message:
"Paket 'viridis' wurde unter R Version 4.2.3 erstellt"
Lade nötiges Paket: viridisLite



## Time averaged field strength density plots

In [2]:
plot_raster_field_strength <- function(max_x, max_y, values, name, charges) {
r <- raster(xmn = 0, xmx = max_x, ymn = 0, ymx = max_y, nrows = max_y, ncols = max_x)
r[] <- values
plot(r, col = plasma(256), xlab = "x", ylab = "y", xaxt = "n", yaxt = "n")
title(paste("Time averaged ", name, "-direction field strength plot\n",
	ifelse(charges, " with Polyakov loops", ""), sep = ""))
axis(1, at = seq(from = 0.5, by = 5, to = max_x), labels = seq(from = 0, by = 5, to = max_x))
axis(2, at = seq(from = 0.5, by = 5, to = max_y), labels = seq(from = 0, by = 5, to = max_y))
box(col = "white")
}

In [3]:
results <- read.csv("..\\data\\results.csv", header = TRUE, sep = ",")
results <- results[results$difference_data == "true", ]

entries <- data.frame(list(
	index = c(5363, 5366, 5364, 5365, 5363, 5366, 5364, 5365),
	direction = c(0, 0, 0, 0, 1, 1, 1, 1),
	blind = c(FALSE, TRUE, FALSE, TRUE, FALSE, FALSE, FALSE, FALSE),
	charges = c(FALSE, TRUE, FALSE, TRUE, FALSE, TRUE, FALSE, TRUE)
))

direction_names <- c("x", "y")

for (i in 1:nrow(entries)) {
	index <- entries$index[i]
	blind <- entries$blind[i]
	direction <- entries$direction[i]
	charges <- entries$charges[i]

	max_x <- results$x[results$index == index]
	max_y <- results$y[results$index == index]

	d <- read.csv(paste("..\\data\\plot_data\\difference_", index, "_", direction, ".csv", sep = ""), header = TRUE, sep = ",")

	# Time averaging
	for (x in 0:(max_x - 1)) for (y in 0:(max_y - 1)) {
		d <- rbind(d, c(x, y, -1, mean(d$value[d$x == x & d$y == y])))
	}
	d <- d[d$t == -1, ]

	if (blind) {
		d$value[which.max(d$value)] <- 0
		d$value[which.min(d$value)] <- 0
		d$value[which.max(d$value)] <- 0
		d$value[which.min(d$value)] <- 0
	}

	if (FALSE) {
		d$x_shifted <- 0
		for (i in 1:nrow(d)) {
			d$x_shifted[i] <- d$value[d$x == (d$x - 1) %% max_x]
		}
	}

	d$x <- (d$x + floor(max_x / 3) + 1) %% max_x
	d$y <- (d$y + floor((max_y - 1) / 2)) %% max_y
	d <- d[order(d$y, d$x), ]

	png(filename = paste("plots\\field_strength_", index, "_", direction, ".png", sep = ""),
		width = 1440, height = 1440, type = c("cairo-png"), res = 250)
	plot_raster_field_strength(max_x, max_y, d$value, direction_names[direction + 1], charges)
	dev.off()
}

Warning message in sp::CRS(...):
"sf required for evolution_status==2L"


## Energy density plot



In [4]:
plot_raster_energy <- function(max_x, max_y, values, name, charges) {
r <- raster(xmn = 0, xmx = max_x, ymn = 0, ymx = max_y, nrows = max_y, ncols = max_x)
r[] <- values
plot(r, col = plasma(256), xlab = "x", ylab = "y", xaxt = "n", yaxt = "n")
title(paste("Time averaged energy density plot\n",
	ifelse(charges, " with Polyakov loops", ""), sep = ""))
axis(1, at = seq(from = 0.5, by = 5, to = max_x), labels = seq(from = 0, by = 5, to = max_x))
axis(2, at = seq(from = 0.5, by = 5, to = max_y), labels = seq(from = 0, by = 5, to = max_y))
box(col = "white")
}

In [6]:
results <- read.csv("..\\data\\results.csv", header = TRUE, sep = ",")
results <- results[results$difference_data == "true", ]

entries <- data.frame(list(
	index = c(5363, 5366, 5364, 5365),
	blind = c(FALSE, TRUE, FALSE, TRUE),
	charges = c(FALSE, TRUE, FALSE, TRUE)
))

for (i in 1:nrow(entries)) {
	index <- entries$index[i]
	blind <- entries$blind[i]
	charges <- entries$charges[i]

	max_x <- results$x[results$index == index]
	max_y <- results$y[results$index == index]
	max_t <- results$t[results$index == index]

	dx <- read.csv(paste("..\\data\\plot_data\\difference_", index, "_", 0, ".csv", sep = ""), header = TRUE, sep = ",")
	dy <- read.csv(paste("..\\data\\plot_data\\difference_", index, "_", 1, ".csv", sep = ""), header = TRUE, sep = ",")
	dt <- read.csv(paste("..\\data\\plot_data\\difference_", index, "_", 2, ".csv", sep = ""), header = TRUE, sep = ",")

	dx$value <- dx$value * dx$value
	dy$value <- dy$value * dy$value
	dt$value <- dt$value * dt$value

	d <- dx
	d$value <- 0
	for (i in 1:nrow(d)) {
		x <- d$x[i]
		y <- d$y[i]
		t <- d$t[i]
		d$value[i] <- dx$value[dx$x == x & dx$y == y & dx$t == t] +
			dy$value[dy$x == x & dy$y == y & dy$t == t] -
			dt$value[dx$x == x & dt$y == y & dt$t == t] +
			dx$value[dx$x == (x - 1) %% max_x & dx$y == y & dx$t == t] +
			dy$value[dy$x == x & dy$y == (y - 1) %% max_y & dy$t == t] -
			dt$value[dx$x == x & dt$y == y & dt$t == (t - 1) %% max_t]
	}

	d$value <- d$value / 2

	# Time averaging
	for (x in 0:(max_x - 1)) for (y in 0:(max_y - 1)) {
		d <- rbind(d, c(x, y, -1, mean(d$value[d$x == x & d$y == y])))
	}
	d <- d[d$t == -1, ]

	if (blind & FALSE) {
		d$value[which.max(d$value)] <- 0
		d$value[which.min(d$value)] <- 0
		d$value[which.max(d$value)] <- 0
		d$value[which.min(d$value)] <- 0
	}

	d$x <- (d$x + floor(max_x / 3) + 1) %% max_x
	d$y <- (d$y + floor((max_y - 1) / 2)) %% max_y
	d <- d[order(d$y, d$x), ]

	
	png(filename = paste("plots\\energy_density_", index, "_", direction, ".png", sep = ""),
		width = 1440, height = 1440, type = c("cairo-png"), res = 250)
	plot_raster_energy(max_x, max_y, d$value, direction_names[direction + 1], charges)
	dev.off()
}